### Add a module that accesses strategies to help with the daily analysis. Should include descriptions of accepted strategies and personal strategies. Save the material in a sqllite db. Unspecified version for this one. Probably an elemental version of this in version 1. 
###### Morning ORB, ABCD, VWAP Reversal (Bull Flag and Fallen angel for low float).
###### Late Morning VWAP false break out and VWAP reversal
###### Mid-Day VWAP Moving Average Trend VWAP false breakout
###### Close VWAP MA Trend
### Prototype version will have the features of version 1 but run in Jupyter notebook.
### Version 1 will include the kind of wizard daily review with prompts to get the picture (videos in an unspecified version) and the stops, targets, explanation and analysis. The nature of those will change when this moves to a windowed app
### Version 2 is going to be the summary stuff.
### Version 2.5 will include archive saving of all the input of Trades from DAS (Datess may be an issue as DAS Trades window is time only (I think)

In [1]:
%matplotlib inline
import pandas as pd

In [2]:
import datetime, os
from PIL import Image as PILImage
from PIL import ImageGrab
from journalfiles import JournalFiles
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook

In [3]:
from IPython.core.display import HTML
css = open('data/style-table.css').read() + open('data/style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [4]:
jf = JournalFiles(outdir = 'out', mydevel = True)
jf.inpathfile
jf.outpathfile

'out\\Trades_Thursday_0906.xlsx'

In [5]:
jf.outpathfile

'out\\Trades_Thursday_0906.xlsx'

In [6]:
trades = pd.read_csv(jf.inpathfile)

In [7]:
trades

,Time,Symb,Side,Price,Qty,Route,Broker,Account,Type,Cloid,P / L,Unnamed: 11
0,12:15:35,MU,B,45.3000,25,SMRT,BEST,U2429974,Margin,180906007522.0,0.00,NaN
1,11:38:07,MU,S,45.3200,42,SMRT,ISLA,U2429974,Margin,180906007238.0,1.68,NaN
2,11:38:07,MU,S,45.3200,100,SMRT,NYSE,U2429974,Margin,180906007238.0,4.00,NaN
3,11:38:07,MU,S,45.3200,100,SMRT,BEST,U2429974,Margin,180906007238.0,4.00,NaN
4,11:38:07,MU,S,45.3200,300,SMRT,BEST,U2429974,Margin,180906007238.0,12.00,NaN
5,11:36:33,MU,S,45.3901,180,SMRT,BEST,U2429974,Margin,180906007212.0,19.82,NaN
6,11:35:12,MU,B,45.2800,81,SMRT,ISLA,U2429974,Margin,180906007179.0,0.00,NaN
7,11:35:12,MU,B,45.2800,50,SMRT,ISLA,U2429974,Margin,180906007179.0,0.00,NaN
8,11:35:12,MU,B,45.2800,591,SMRT,ISLA,U2429974,Margin,180906007179.0,0.00,NaN
9,11:32:10,MU,S,45.0215,364,SMRT,BEST,U2429974,Margin,180906007116.0,74.57,NaN


In [8]:
def checkRequiredInputFields(dframe) :
    RequiredFields = ['Time', 'Symb', 'Side', 'Price', 'Qty', 'P / L']
    ActualFields=dframe.columns
    if set (RequiredFields) <= (set (ActualFields)) :
        print("got it")
    else :
        err='You are missing some fields in your input file:\n'
        err += str((set(RequiredFields) - set(ActualFields)))
        raise ValueError(err)


In [9]:
def createDf(dframe, numRow) :
    ''' Creates a new DataFrame with  the length numRow. Each cell is filled with empty string '''

    ll=list()
    r=list()
    for i in range(len(dframe.columns)) :
        r.append('')
        
    for i in range(numRow) :
        ll.append(r)
    newdf= pd.DataFrame(ll, columns=dframe.columns)

    return newdf

In [10]:
def addRows(dframe, numRow) :
    newdf= createDf(dframe, numRow)
        
    
    dframe = dframe.append(newdf, ignore_index=True, sort=False)
    
    return dframe

    

### Make sure the hour string is 0 padded.  Should probably change these to data types. 

### Todo- This, and the duration, will get more complex with the IB report format. The html report uses a long timedate format

In [11]:
def zeroPadTimeStr(dframe, timeHeading) :
    '''Guarantee that the time format xx:xx:xx'''
    

    for i, row in dframe.iterrows():
        tm = row[timeHeading]
        tms=tm.split(":")
        if int(len(tms[0]) < 2) :
            if tms[0].startswith("0") == False :
                tm= "0" + tm
                dframe.at[i, timeHeading] = tm
    return dframe
                

### Todo. Doctor an input csv file to include fractional numer of shares for testing. Make it more modular by checking for 'HOLD'. It might be useful in a windowed version with menus to do things seperately.

In [12]:
def mkShortsNegative(dframe, side, qty) :
    ''' Fix the shares sold to be negative values. Currently assuming DAS values 'B', 'S', 'SS' for 'Side'. 
    '''
    for i, row in dframe.iterrows():
        if row[side] != 'B' and row[qty] > 0:
            dframe.at[i, qty] = ((dframe.at[i, qty]) * -1)
    return dframe

### def getListTickerDF(df) will take a dataframe that includes tickers in column 'Symb' and returns a python list of DataFrames , 1 for each ticker.

In [13]:
def getListTickerDF(dframe, tickCol = 'Symb') :
    ldf_tick = list()
    for ticker in dframe[tickCol].unique() :
        ldf = dframe[dframe[tickCol] == ticker]
        ldf_tick.append(ldf)
    return ldf_tick

In [14]:

def getOvernightTrades(dframe, tickCol='Symb', qtyCol='Qty') :
    ''' getOvernightTrades(dframe) takes the DataFrame and returns a list of lists 
        (symb, qty) of trades that have overnight shares. '''
    
    ldf_tick = getListTickerDF(dframe, tickCol)
    overnightTrade = list()
    for ticker in ldf_tick :
        if ticker[qtyCol].sum() != 0 :
            overnightTrade.append([ticker[tickCol].unique()[0], ticker[qtyCol].sum(), 0, 0])
    return overnightTrade

### Note that this does not yet include those shares that are held before the days trading began. Redo this to remake the list of data frames from the Symbols of Swing List then make a list of data frams that  excludes those then merge them together

In [15]:
def askUser(st, question, ix, default) :
    while True :
        try :
            response = input(question)
            if len(response) < 1 :
                response = default
            else :
                response = int(response)
        except Exception as ex:
            print(ex)
            print("please enter a number")
            continue
       
        st[ix] = response
        return st


In [16]:
# for i in range(len(swingTrade)) :
def figureOvernightTransactions() :
    swingTrade = getOvernightTrades(trades)
    for i in range(len(swingTrade)) :
        tryAgain =  True
        while tryAgain == True :

            xticker = 0      #Candy coaters
            xbalance = 1
            xbefore = 2
            xafter = 3
            print(swingTrade[i])
            print ("There is an unbalanced amount of shares of {0} in the amount of {1}".format(swingTrade[i][xticker], swingTrade[i][xbalance]))

            question = "How many shares of {0} are you holding now? (Enter for {1})".format(swingTrade[i][xticker], swingTrade[i][xbalance])
            swingTrade[i] = askUser(swingTrade[i], question, xafter, swingTrade[i][xbalance])

            if swingTrade[i][xafter] != swingTrade[i][xbalance]:

                difference = swingTrade[i][xbalance] - swingTrade[i][xafter]
                statement = "There is now a prior unbalanced amount of shares of {0} in the amount of {1}"
                print(statement.format(swingTrade[i][xticker], difference))
                question = "How many shares of {0} were you holding before?".format(swingTrade[i][xticker])
                swingTrade[i] = askUser(swingTrade[i], question, xbefore, difference)

            unaccounted = swingTrade[i][xbefore] + swingTrade[i][xafter] - swingTrade[i][xbalance]
            if unaccounted == 0 :
                print("That works.")
                tryAgain = False
            else :
                print()
                print("There are {1} unaccounted for shares in {0}".format(swingTrade[i][xticker], unaccounted))
                print()
                print("That does not add up. Starting over ...")
                print()
                print ("Prior to reset version ", i, swingTrade)
                swingTrade[i] = getOvernightTrades(trades)[i]
                print ("reset version ", i, swingTrade)
    return swingTrade

In [17]:
def insertOvernightRow(dframe, st, time='Time', symbol='Symb', side='Side', price='Price', qty='Qty', acct="Account", PL='P / L') :
    newdf = createDf(trades, 0)
    
    for ldf in getListTickerDF(dframe) :
        found=False
        for ticker, balance, before, after in swingTrade:
            if ticker == ldf.Symb.unique()[0] :
                print ("Got {0} with the balance {1}, before {2} and after {3}". format (ticker, balance, before, after))
                if before != 0 :
                    newldf = createDf(dframe, 1)
                    print("length:   ", len(newldf))
                    for i, row in newldf.iterrows():

                        if i == len(newldf) -1 :
                            print("Though this seems unnecessary it will make it more uniform ")
                            newldf.at[i, time] = '00:00:01'
                            newldf.at[i, symbol] = ticker
                            if before > 0 :
                                newldf.at[i, side] = "HOLD+"
                            else :
                                newldf.at[i, side] = "HOLD-"
                            newldf.at[i, price] = 0
                            newldf.at[i, qty] = before
                            newldf.at[i, acct] = 'ZeroSubstance'
                            newldf.at[i, PL] = 0
                            
                            ldf = newldf.append(ldf, ignore_index = True)
                        break #This should be unnecessary as newldf should always be the length of 1 here
                if after != 0 :
                    print("Are we good?")
                    ldf = addRows(ldf, 1)
        
                    for i, row in ldf.iterrows():

                        if i == len(ldf) -1 :
                            ldf.at[i, time] = '23:59:59'
                            ldf.at[i, symbol] = ticker
                            if balance > 0 :
                                ldf.at[i, side] = "HOLD+"
                            else :
                                ldf.at[i, side] = "HOLD-"
                            ldf.at[i, price] = 0
                            ldf.at[i, qty] = after
                            ldf.at[i, acct] = 'ZeroSubstance'
                            ldf.at[i, PL] = 0
        
        newdf = newdf.append(ldf, ignore_index = True, sort = False)
    return newdf

In [18]:
def writeShareBalance(dframe) :
    prevBal = 0
    for i, row in dframe.iterrows():
        qty = (dframe.at[i,'Qty'])
        if row['Side'].startswith("HOLD") :
            print("got it at ", qty)
            qty = qty * -1
        newBalance = qty + prevBal

        dframe.at[i,'Balance'] = newBalance 
        prevBal = newBalance    
#     return nt
    return dframe

In [19]:
def addStartTime(dframe) :
    newTrade = True
    for i, row in dframe.iterrows():
#         print(i, row['Time'])
        if newTrade :
            if row['Side'].startswith('HOLD') and i < len(dframe):
                
                oldTime = dframe.at[i+1, 'Time']
#                 print("     :Index: {0},  Side: {1}, Time{2}, setting {3}".format(i, row['Side'], row['Time'], oldTime))
                dframe.at[i, 'Start'] = oldTime

            else :
                oldTime = dframe.at[i, 'Time']
#                 print("     :Index: {0},  Side: {1}, Time{2}, setting {3}".format(i, row['Side'], row['Time'], oldTime))
                dframe.at[i, 'Start'] = oldTime
                
                
            newTrade = False
        else :
#             print("      Finally :Index: {0},  Side: {1}, Time{2}, setting {3}".format(i, row['Side'], row['Time'], oldTime))
            dframe.at[i, 'Start'] = oldTime
        if row['Balance'] == 0 :
#             print("       setting newTrade")
            newTrade = True
    return dframe
        

## Now that we have Trade index and start times everything else in this file should use filters nt[nt.Tindex == "Trade 1"] . This way each trade can be treated as an object that includes a variable number of transactions.

In [20]:
def addTradeIndex(dframe) :

    TCount = 1
    prevEndTrade = -1

    for i, row in dframe.iterrows():
        if len(row['Symb']) < 1 :
            break
        tradeIndex = "Trade " + str(TCount)
        if prevEndTrade == 0:
            TCount = TCount + 1
            prevEndTrade = -1
        tradeIndex = "Trade " + str(TCount)
        dframe.at[i,'Tindex'] = tradeIndex 
        if row['Balance'] == 0 :
            prevEndTrade = 0
    numTrades = TCount
    print(numTrades)        
    return dframe

### This will blow up from wrong types if there is some kind of anomaly. The blank rows in Sum col are str and the filled rows are float. Don't fix it until it blows up. (or you need change this 'now' to production code for some reason) That way we have some case to fix.

###### nt['P / L'].sum()
###### nt['P / L'].dtype

In [21]:
def addTradePL (dframe) :
    tradeTotal = 0.0
    for i, row in dframe.iterrows():
        if row['Balance'] != 0 :
            tradeTotal = tradeTotal + row['P / L']
        else :
            sumtotal = tradeTotal + row['P / L']
            dframe.at[i, 'Sum'] = sumtotal
            tradeTotal = 0
    return dframe


In [22]:
def addTradeDuration(dframe) :
    
    tradeTotal = 0.0
    for i, row in dframe.iterrows():
        if row['Balance'] == 0 :
            timeEnd = row['Time']
            timeStart = row['Start']
            end=timeEnd.split(":")
            start=timeStart.split(":")
            diff = datetime.datetime(1,1,1,int(end[0]), int(end[1]), int(end[2])) - datetime.datetime(1,1,1,int(start[0]), int(start[1]), int(start[2]))
            dframe.at[i,'Duration'] = diff
    return dframe


In [23]:
def addTradeName(dframe) :
    for i, row in dframe.iterrows():
        longShort = " Long"
        if row['Balance'] == 0 :
            if row['Side'] == 'B' :
                longShort = " Short"
            dframe.at[i, 'Name'] = row['Symb'] + longShort
    return dframe



In [24]:
# Note that .sum() should work on this but it failed when I tried it.
def addSummaryPL(dframe) :
    
    count=0
    tot=0.0
    tot2 = 0.0
    for i, row in dframe.iterrows():
        count=count+1
        if count < len(dframe) :
            tot=tot+row['P / L']
            if row['Balance'] == 0 :
 
                tot2 = tot2 + row['Sum']
            if count == len(dframe) -1 :
                lastCol = row['P / L']

                print("Last col?", row['P / L'])
   
        else :
            dframe.at[i, 'P / L'] = tot
            dframe.at[i, 'Sum'] = tot2

    if lastCol > 0:

        print('''
        Some shares are unaccounted for. Please send the original csv file to the developer in 
        order to fix ths issue in the software. Please remove the account number  or change its value
        to anything else.
        ''')
    return dframe


In [25]:
# Adjust size to keep the aspect ration
# The actual version should calculate the height based
# no the number of cells between entries. That 
# will be a number chosen by the user (with constraints) 
# pixels per cell height is about 20.238095238095237
# The default size (425) is height about 21 unaltered excel cells

def adjustSizeByHeight(sz, newHeight=425) :
    w,h = sz
    newWidth = int((newHeight/h) * w)
    newheight = int(newHeight)
    return(newWidth, newHeight)

In [26]:
def getPilImageFromClipboard() :
    for i in range (5) :
        response = input("Are you ready? ")
        if response.lower().startswith('y') : 
            im = ImageGrab.grabclipboard()
            if im is None :
                print("Failed to get an image. Please select and copy an image")
            else :
                return im
        else :
            if response.lower().startswith('q') :
                return None
    print("Moving on")
# img = getPilImageFromClipboard()
# img

In [27]:
# orig: an image file name
# Returns a tuple(newFileName, extension)
#     changes jpg to jpeg to appease PIL.save()

def getResizeName(orig) :
    x=os.path.splitext(orig)
    if (len (x[1]) < 4 ) :
        print("please provide an image name with an image extension in its name. e.g 'png', jpg', etc")
    newName = x[0] + '_resize'
    if 'jpg' in x[1].lower() :
        newName += '.jpeg'
    else :
        newName += x[1]
    newName = os.path.join(jf.outdir, newName)
    return (newName, os.path.splitext(newName)[1][1:])

# getResizeName('fred.png')

In [28]:
jf.outdir

'out'

In [29]:
getResizeName('fred.jpg')

('out\\fred_resize.jpeg', 'jpeg')

In [30]:
# dframe contains the transactions of a single trade.  Single trade ends when the balance of shares is 0
# Return value is a string 'Long' or 'Short'
def getLongOrShort(dframe) :
    tsx = dframe[dframe.Balance == 0]

    
    if len(tsx) != 1 :
        return None
    if str(tsx.Side) == 'B' or str(tsx.Side) == 'Hold-':
        return 'Short'    
    else :
        return 'Long'
    

    

###### dframe contains the transactions of a single trade.  longOrShort needs to be a str 'Long' or 'Short'. Will raise an exception if they are not Returns a tuple of dataFrames (entries, exits)-- not ready for prime time
def getEntriesAndExits(dframe,longOrShort) :
    if longOrShort.lower() not in ['long', 'short'] :
        #Programming error if we are here
        raise NameError('getEntriesAndExits requires a parameter of either Long or Short. %s is not acceptible' % longOrShort)
        return None
    f_ent, f_ext = ('B', 'S') if longOrShort == 'Long' else ('S', 'B')
    ent = tdf[tdf.Side.str.startswith(f_ent)]
    ext = tdf[tdf.Side.str.startswith(f_ext)]
    return ent, ext
getEntriesAndExits(tdf, 'Short')[0]

In [31]:
checkRequiredInputFields(trades)

got it


In [32]:
trades = zeroPadTimeStr(trades, 'Time')

In [33]:
trades

,Time,Symb,Side,Price,Qty,Route,Broker,Account,Type,Cloid,P / L,Unnamed: 11
0,12:15:35,MU,B,45.3000,25,SMRT,BEST,U2429974,Margin,180906007522.0,0.00,NaN
1,11:38:07,MU,S,45.3200,42,SMRT,ISLA,U2429974,Margin,180906007238.0,1.68,NaN
2,11:38:07,MU,S,45.3200,100,SMRT,NYSE,U2429974,Margin,180906007238.0,4.00,NaN
3,11:38:07,MU,S,45.3200,100,SMRT,BEST,U2429974,Margin,180906007238.0,4.00,NaN
4,11:38:07,MU,S,45.3200,300,SMRT,BEST,U2429974,Margin,180906007238.0,12.00,NaN
5,11:36:33,MU,S,45.3901,180,SMRT,BEST,U2429974,Margin,180906007212.0,19.82,NaN
6,11:35:12,MU,B,45.2800,81,SMRT,ISLA,U2429974,Margin,180906007179.0,0.00,NaN
7,11:35:12,MU,B,45.2800,50,SMRT,ISLA,U2429974,Margin,180906007179.0,0.00,NaN
8,11:35:12,MU,B,45.2800,591,SMRT,ISLA,U2429974,Margin,180906007179.0,0.00,NaN
9,11:32:10,MU,S,45.0215,364,SMRT,BEST,U2429974,Margin,180906007116.0,74.57,NaN


In [34]:
trades = trades.sort_values(['Symb', 'Time'])

In [35]:
trades

,Time,Symb,Side,Price,Qty,Route,Broker,Account,Type,Cloid,P / L,Unnamed: 11
18,11:03:22,ARWR,SS,21.4400,938,SMRT,NaN,TRIB5621,Short,AUTO,0.00,NaN
17,11:05:07,ARWR,B,21.3300,469,SMRT,NaN,TRIB5621,Margin,AUTO,51.59,NaN
16,11:06:49,ARWR,B,21.1000,117,SMRT,NaN,TRIB5621,Margin,AUTO,39.78,NaN
15,11:08:55,ARWR,B,21.1000,88,SMRT,NaN,TRIB5621,Margin,AUTO,29.92,NaN
14,11:12:52,ARWR,B,21.3500,264,SMRT,NaN,TRIB5621,Margin,AUTO,23.76,NaN
21,09:37:50,MU,SS,47.3700,633,SMRT,NaN,TRIB5621,Short,AUTO,0.00,NaN
20,09:38:06,MU,SS,47.4200,442,SMRT,NaN,TRIB5621,Short,AUTO,0.00,NaN
19,09:40:08,MU,B,47.2300,268,SMRT,NaN,TRIB5621,Margin,AUTO,43.03,NaN
25,09:40:31,MU,B,47.1000,201,SMRT,NaN,TRIB5621,Margin,AUTO,58.41,NaN
24,09:40:55,MU,B,47.1800,606,SMRT,NaN,TRIB5621,Margin,AUTO,127.60,NaN


In [36]:
trades = mkShortsNegative(trades, 'Side', 'Qty')

In [37]:
trades

,Time,Symb,Side,Price,Qty,Route,Broker,Account,Type,Cloid,P / L,Unnamed: 11
18,11:03:22,ARWR,SS,21.4400,-938,SMRT,NaN,TRIB5621,Short,AUTO,0.00,NaN
17,11:05:07,ARWR,B,21.3300,469,SMRT,NaN,TRIB5621,Margin,AUTO,51.59,NaN
16,11:06:49,ARWR,B,21.1000,117,SMRT,NaN,TRIB5621,Margin,AUTO,39.78,NaN
15,11:08:55,ARWR,B,21.1000,88,SMRT,NaN,TRIB5621,Margin,AUTO,29.92,NaN
14,11:12:52,ARWR,B,21.3500,264,SMRT,NaN,TRIB5621,Margin,AUTO,23.76,NaN
21,09:37:50,MU,SS,47.3700,-633,SMRT,NaN,TRIB5621,Short,AUTO,0.00,NaN
20,09:38:06,MU,SS,47.4200,-442,SMRT,NaN,TRIB5621,Short,AUTO,0.00,NaN
19,09:40:08,MU,B,47.2300,268,SMRT,NaN,TRIB5621,Margin,AUTO,43.03,NaN
25,09:40:31,MU,B,47.1000,201,SMRT,NaN,TRIB5621,Margin,AUTO,58.41,NaN
24,09:40:55,MU,B,47.1800,606,SMRT,NaN,TRIB5621,Margin,AUTO,127.60,NaN


In [38]:
swingTrade = getOvernightTrades(trades)

In [39]:
swingTrade

[['MU', 25, 0, 0]]

In [40]:
swingTrade = figureOvernightTransactions()

['MU', 25, 0, 0]
There is an unbalanced amount of shares of MU in the amount of 25
How many shares of MU are you holding now? (Enter for 25)
That works.


In [41]:
swingTrade

[['MU', 25, 0, 25]]

In [42]:

listdf = insertOvernightRow(trades, swingTrade)

Got MU with the balance 25, before 0 and after 25
Are we good?


In [43]:
listdf

,Time,Symb,Side,Price,Qty,Route,Broker,Account,Type,Cloid,P / L,Unnamed: 11
0,11:03:22,ARWR,SS,21.44,-938,SMRT,NaN,TRIB5621,Short,AUTO,0,NaN
1,11:05:07,ARWR,B,21.33,469,SMRT,NaN,TRIB5621,Margin,AUTO,51.59,NaN
2,11:06:49,ARWR,B,21.1,117,SMRT,NaN,TRIB5621,Margin,AUTO,39.78,NaN
3,11:08:55,ARWR,B,21.1,88,SMRT,NaN,TRIB5621,Margin,AUTO,29.92,NaN
4,11:12:52,ARWR,B,21.35,264,SMRT,NaN,TRIB5621,Margin,AUTO,23.76,NaN
5,09:37:50,MU,SS,47.37,-633,SMRT,NaN,TRIB5621,Short,AUTO,0,NaN
6,09:38:06,MU,SS,47.42,-442,SMRT,NaN,TRIB5621,Short,AUTO,0,NaN
7,09:40:08,MU,B,47.23,268,SMRT,NaN,TRIB5621,Margin,AUTO,43.03,NaN
8,09:40:31,MU,B,47.1,201,SMRT,NaN,TRIB5621,Margin,AUTO,58.41,NaN
9,09:40:55,MU,B,47.18,606,SMRT,NaN,TRIB5621,Margin,AUTO,127.6,NaN


In [44]:
lbls = ['Tindex', 'Start', 'Time', 'Symb', 'Side', 'Price', 'Qty','Balance', 'Account', "P / L", 'Sum', 'Duration', 'Name']

In [45]:
for l in lbls :
    if l not in listdf.columns :
        listdf[l] = ''
# trades.columns
newTrades = listdf[lbls]
newTrades.copy()
# newTrades.columns

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
0,,,11:03:22,ARWR,SS,21.44,-938,,TRIB5621,0,,,
1,,,11:05:07,ARWR,B,21.33,469,,TRIB5621,51.59,,,
2,,,11:06:49,ARWR,B,21.1,117,,TRIB5621,39.78,,,
3,,,11:08:55,ARWR,B,21.1,88,,TRIB5621,29.92,,,
4,,,11:12:52,ARWR,B,21.35,264,,TRIB5621,23.76,,,
5,,,09:37:50,MU,SS,47.37,-633,,TRIB5621,0,,,
6,,,09:38:06,MU,SS,47.42,-442,,TRIB5621,0,,,
7,,,09:40:08,MU,B,47.23,268,,TRIB5621,43.03,,,
8,,,09:40:31,MU,B,47.1,201,,TRIB5621,58.41,,,
9,,,09:40:55,MU,B,47.18,606,,TRIB5621,127.6,,,


In [46]:
nt = newTrades.sort_values(['Symb', 'Time'])

In [48]:
nt

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
0,,,11:03:22,ARWR,SS,21.44,-938,,TRIB5621,0,,,
1,,,11:05:07,ARWR,B,21.33,469,,TRIB5621,51.59,,,
2,,,11:06:49,ARWR,B,21.1,117,,TRIB5621,39.78,,,
3,,,11:08:55,ARWR,B,21.1,88,,TRIB5621,29.92,,,
4,,,11:12:52,ARWR,B,21.35,264,,TRIB5621,23.76,,,
5,,,09:37:50,MU,SS,47.37,-633,,TRIB5621,0,,,
6,,,09:38:06,MU,SS,47.42,-442,,TRIB5621,0,,,
7,,,09:40:08,MU,B,47.23,268,,TRIB5621,43.03,,,
8,,,09:40:31,MU,B,47.1,201,,TRIB5621,58.41,,,
9,,,09:40:55,MU,B,47.18,606,,TRIB5621,127.6,,,


In [49]:
nt = writeShareBalance(nt)

got it at  25


In [50]:
nt

,Tindex,Start,Time,Symb,Side,Price,Qty,Balance,Account,P / L,Sum,Duration,Name
0,,,11:03:22,ARWR,SS,21.44,-938,-938,TRIB5621,0,,,
1,,,11:05:07,ARWR,B,21.33,469,-469,TRIB5621,51.59,,,
2,,,11:06:49,ARWR,B,21.1,117,-352,TRIB5621,39.78,,,
3,,,11:08:55,ARWR,B,21.1,88,-264,TRIB5621,29.92,,,
4,,,11:12:52,ARWR,B,21.35,264,0,TRIB5621,23.76,,,
5,,,09:37:50,MU,SS,47.37,-633,-633,TRIB5621,0,,,
6,,,09:38:06,MU,SS,47.42,-442,-1075,TRIB5621,0,,,
7,,,09:40:08,MU,B,47.23,268,-807,TRIB5621,43.03,,,
8,,,09:40:31,MU,B,47.1,201,-606,TRIB5621,58.41,,,
9,,,09:40:55,MU,B,47.18,606,0,TRIB5621,127.6,,,


In [ ]:
nt = addStartTime(nt)

In [ ]:
nt

In [ ]:
nt = nt.sort_values(['Start', 'Time'])

In [ ]:
nt

In [ ]:
nt = addTradeIndex(nt)

In [ ]:
nt

In [ ]:
nt = addTradePL(nt)

In [ ]:
nt

In [ ]:
nt = addTradeDuration(nt)

In [ ]:
nt

In [ ]:
nt = addTradeName(nt)

In [ ]:
nt

In [ ]:
nt=addRows(nt,1)

In [ ]:
nt = addSummaryPL(nt)
# nt = addSummaryPL(nt)

In [ ]:
nt

In [ ]:
# adjustSizeByHeight ((1024,2037))

In [ ]:
# Request a clipboard copy of an image. Resze it to newSize height. Save it with a new name. Return the name. Hackiness lives
# until I figure how to create a proper openpyxl Image object from a PIL Image object that doesn't make the Workbook puke.
def getAndResizeImage (newSize) :
    try :
#         pilImage= PILImage.open(imgName)
        pilImage = getPilImageFromClipboard() 
        newSize = adjustSizeByHeight(pilImage.size)
        pilImage = pilImage.resize(newSize, PILImage.ANTIALIAS)
        resizeName, ext = getResizeName(imgName)
        pilImage.save(resizeName, ext)

    except IOError as e :
        print("An exception occured '%s'" % e)
    return resizeName

In [ ]:
# Now  we are going to add each trade and insert space to put in pictures with circles and arrows and 
# paragraph on the back of each one to be used as evidence against you in a court of law (or court of bb opionion)
insertsize=25
dframe = nt
ldf = list()
count = 1
while True :
    tradeStr = "Trade " + str(count)
    count = count + 1
    tdf = dframe[dframe.Tindex == tradeStr]
    if len(tdf) > 0 :
        ldf.append(tdf)
    else :
        break
len(ldf)

dframe = addRows(dframe, 2)

In [ ]:
# ldf


### TODO The Hold is not operating correctly. This method is writen like I intend it to function

In [ ]:
entries = list()
exits= list()
for tdf in ldf:
    longOrShort = getLongOrShort(tdf)
    
    ent = tdf[tdf.Side == 'B']
    ext = tdf[tdf.Side.str.startswith('S')]
    entries.append(ent)
    exits.append(ext)
    print(getLongOrShort(tdf))
        
    break


In [ ]:
for tdf in ldf:
    print (tdf.Tindex.unique()[0].replace(' ','') + '.jpeg')

In [ ]:
s = 'Trade 1'.replace(" ", "") + '.jpeg'
s

In [ ]:
ldf[1]

In [ ]:
print(ldf[0].Tindex.unique()[0])
print (ldf[0].Name.unique()[-1])
print(ldf[0].Start.unique()[0])
print (ldf[0].Duration.unique()[-1])

The Dataframe grabshere rely far too much on proper placement by this program. This is no longer DataFrame like data. At this point, the trades will be encapsulated in objects.

In [ ]:
imageLocation = list()
newdf = createDf(dframe,  10)
topMargin = 10
dframe = newdf.append(dframe, ignore_index = True)
for tdf in ldf :

    # TODO handle empty string in the tdf
    imageLocation.append([len(tdf) + len(dframe) + 2, 
                          tdf.Tindex.unique()[0].replace(' ', '') + '.jpeg',
                          tdf.Name.unique()[-1],
                          tdf.Start.unique()[-1],
                        tdf.Duration.unique()[-1]])
    print(len(tdf) + len(dframe) + 2)

    dframe = dframe.append(tdf, ignore_index = True)
    dframe = addRows(dframe, insertsize)
    print(count, ": ", len(dframe))

len(dframe)

In [ ]:
for x in imageLocation :
    print(x)

In [ ]:
# for tdf in ldf :
#     dframe = dframe.append(tdf, ignore_index = True)
#     dframe = addRows(dframe, insertsize)
#     print(count, ": ", len(dframe))
    
nt = dframe

In [ ]:
# nt

#####  TODO - Catch Permission Denied exception and inform the user

In [ ]:

wb = Workbook()
ws = wb.active

for r in dataframe_to_rows(nt, index=False, header=False):
    ws.append(r)

In [ ]:
for name, cell  in zip(nt.columns, ws[topMargin]) :
    cell.value = name
#     print(name, cell.value)

In [ ]:
for c in ws[10] :
    print (c.value)

In [ ]:
nt

In [ ]:
for i in imageLocation :
#     print("Copy an image into the clipboard for {0}, beginning {1} and lasting {3}".format(i[0], i[1], [2]))
    print('Copy an image into the clipboard for {0} beginning {1}, and lasting {2}'.format(i[2], i[3], i[4]))

In [ ]:
imgName = 'tempImage.jpg'
for loc in imageLocation :
    try :
        pilImage = getPilImageFromClipboard() 
        newSize = adjustSizeByHeight(pilImage.size)
        pilImage = pilImage.resize(newSize, PILImage.ANTIALIAS)
        print('Copy an image into the clipboard for {0} beginning {1}, and lasting {2}'.format(loc[1], loc[2], loc[3]))
        resizeName, ext = getResizeName(loc[1])
        print (resizeName)
        pilImage.save(resizeName, ext)
        img = openpyxl.drawing.image.Image(resizeName)
        cellname = 'E' + str(loc[0])
        ws.add_image(img, cellname)
        print(cellname)
            
            
            
            
            
            
            
            

    except IOError as e :
        print("An exception occured '%s'" % e)
        
            


In [ ]:
outFile


In [ ]:
wb.save(outFile)

In [ ]:
for c in ws[10]:
    print(c.value)

In [ ]:
outFile

In [ ]:
# name=datetime.date.today().strftime("Trades_%A_%m%d.csv")

# nt.to_csv(name, index=False)
# nt.to_excel("AddingPictures.xlsx", index=False)
namexl

In [ ]:
tdir='C:\\trader\\journal\\_08_August\\Week_4\\_0821_Tuesday\\'
# tdir = "out"
# outfile=tdir+ name
# nt.to_csv(outfile)


In [ ]:
jname=os.path.join(tdir,namexl)
jname

In [ ]:
nt.to_excel(jname, index=False)

In [ ]:
jname